In [18]:
from astropy.io import fits
import math
import pandas as pd
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import matplotlib
import random
import scipy.stats as stats
import scipy.special as special
import emcee
import corner
import math


hdu=fits.open( 'outerdisk_sample.fits' )
hdu.info()
print(hdu[1].header)
r=hdu[1].data.field('rgc')
rv=hdu[1].data.field('rv')
rv_err=hdu[1].data.field('rv_err')
nansr = np.isnan(r)
rv = np.delete(rv, np.where(nansr))
rv_err= np.delete(rv_err, np.where(nansr))
r= np.delete(r, np.where(nansr))
m=np.mean(rv)
print(m)

def wudibaolongzhansheng(x,rv,rv_err,r):
    a=x[0]
    hr=x[1]
    dr = abs((r - 8.34) * 0.2)
    r0=stats.norm.rvs(loc=r, scale=dr, size=len(r))
    sig2=a*np.exp(-r/hr)
    sig2z=sig2+(rv_err**2)
    L = np.sum(-((rv-6.17)**2)/sig2z-0.5*np.log(sig2z))
    return L



Filename: outerdisk_sample.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU      16   (10006,)   uint8   
  1  Joined        1 BinTableHDU     75   23983R x 18C   [E, E, E, E, E, E, D, D, D, D, D, D, D, D, D, D, D, L]   
XTENSION= 'BINTABLE'           / binary table extension                         BITPIX  =                    8 / 8-bit bytes                                    NAXIS   =                    2 / 2-dimensional table                            NAXIS1  =                  113 / width of table in bytes                        NAXIS2  =                23983 / number of rows in table                        PCOUNT  =                    0 / size of special data area                      GCOUNT  =                    1 / one data group                                 TFIELDS =                   18 / number of columns                              EXTNAME = 'Joined  '           / table name                                     TTYPE1  =

In [19]:
nwalker = 50
ndim = 2
p0 = np.zeros((nwalker,ndim))
pos = np.random.randn(50, ndim)
p0[:,0] = np.random.rand(nwalker)
p0[:,1] = np.random.rand(nwalker)
sampler = emcee.EnsembleSampler(nwalker, ndim, wudibaolongzhansheng,args=[rv,rv_err,r])
pos,prob,stat = sampler.run_mcmc(p0,100)
sampler.reset()
sampler.run_mcmc(pos,1000)
samples = sampler.chain[:, :, :].reshape((-1, ndim))
fig = corner.corner(samples,\
    labels=[r'$d$',r'$d_0$'],\
        quantiles=[0.16, 0.5, 0.84],\
        show_titles=True, \
        title_kwargs={"fontsize": 12})

/tmp/ipykernel_50088/4034545495.py:36: RuntimeWarning: invalid value encountered in log
  L = np.sum(-((rv-6.17)**2)/sig2z-0.5*np.log(sig2z))
/tmp/ipykernel_50088/4034545495.py:34: RuntimeWarning: overflow encountered in exp
  sig2=a*np.exp(-r/hr)


ValueError: Probability function returned NaN

In [17]:
dim = 2
pos =  np.random.randn(20, dim)
nwalkers, ndim = pos.shape
sampler = emcee.EnsembleSampler(nwalkers, ndim, wudibaolongzhansheng, args=[rv,rv_err,r])

sampler.run_mcmc(pos,3000, progress=True);

emcee: Exception while calling your likelihood function:
  params: [ 0.51253835 -0.3802547 ]
  args: [array([13.95, 20.17, 12.98, ...,  5.47, 57.62,  3.09], dtype='>f4'), array([15.41, 12.  , 17.71, ..., 15.1 , 14.99, 11.84], dtype='>f4'), array([ 9.12892758, 13.07438896, 10.03329427, ..., 17.07729439,
       11.26720201, 11.74873069], dtype='>f8')]
  kwargs: {}
  exception:


Traceback (most recent call last):
  File "/home/sralus/.conda/envs/astrostatistics/lib/python3.11/site-packages/emcee/ensemble.py", line 624, in __call__
    return self.f(x, *self.args, **self.kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/tmp/ipykernel_50088/921217932.py", line 36, in wudibaolongzhansheng
    L = np.sum(-((rv-6.17)**2)/sig2z-0.5*np.log(sig2z))
               ~~~~~~~~~~~~~~~^~~~~~
ValueError: operands could not be broadcast together with shapes (2,) (23624,) 


ValueError: operands could not be broadcast together with shapes (2,) (23624,) 